In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from math import sqrt


In [10]:
df=pd.read_excel('19MDVRP Problem Sets.xlsx')
df

,Problem,Customer Number,x coordinate,y coordinate,Number of Depots,Depot x coordinate,Depot y coordinate
0,1.0,1,37,52,1.0,20.0,20.0
1,NaN,2,49,49,2.0,30.0,40.0
2,NaN,3,52,64,3.0,50.0,30.0
3,NaN,4,20,26,4.0,60.0,50.0
4,NaN,5,40,30,NaN,NaN,NaN
5,NaN,6,21,47,NaN,NaN,NaN
6,NaN,7,17,63,NaN,NaN,NaN
7,NaN,8,31,62,NaN,NaN,NaN
8,NaN,9,52,33,NaN,NaN,NaN
9,NaN,10,51,21,NaN,NaN,NaN


In [10]:
depots_df =  df.dropna(subset=['Number of Depots'])[['Depot x coordinate', 'Depot y coordinate']].drop_duplicates().reset_index(drop=True)

In [12]:
depots = []
for i, row in depots_df.iterrows():
    depots.append({
        "id": i,
        "x": int(row['Depot x coordinate']),
        "y": int(row['Depot y coordinate']),
    })

# استخراج بيانات العملاء (كل الصفوف اللي رقم العميل موجود)
customers_df = df[['Customer Number', 'x coordinate', 'y coordinate']].dropna()

# تحويل لمصفوفة dict للعملاء مع id (صفر-based index) و demand 0 لأن غير معرف
customers = []
for i, row in customers_df.iterrows():
    customers.append({
        "id": int(row['Customer Number']) - 1,  # نحول ل zero-based index
        "x": int(row['x coordinate']),
        "y": int(row['y coordinate']),
        "demand": 0  # ممكن تغير حسب بياناتك لاحقاً
    })

# طباعة النتائج
print("depots =", depots)
print("customers =", customers)

depots = [{'id': 0, 'x': 20, 'y': 20}, {'id': 1, 'x': 30, 'y': 40}, {'id': 2, 'x': 50, 'y': 30}, {'id': 3, 'x': 60, 'y': 50}]
customers = [{'id': 0, 'x': 37, 'y': 52, 'demand': 0}, {'id': 1, 'x': 49, 'y': 49, 'demand': 0}, {'id': 2, 'x': 52, 'y': 64, 'demand': 0}, {'id': 3, 'x': 20, 'y': 26, 'demand': 0}, {'id': 4, 'x': 40, 'y': 30, 'demand': 0}, {'id': 5, 'x': 21, 'y': 47, 'demand': 0}, {'id': 6, 'x': 17, 'y': 63, 'demand': 0}, {'id': 7, 'x': 31, 'y': 62, 'demand': 0}, {'id': 8, 'x': 52, 'y': 33, 'demand': 0}, {'id': 9, 'x': 51, 'y': 21, 'demand': 0}, {'id': 10, 'x': 42, 'y': 41, 'demand': 0}, {'id': 11, 'x': 31, 'y': 32, 'demand': 0}, {'id': 12, 'x': 5, 'y': 25, 'demand': 0}, {'id': 13, 'x': 12, 'y': 42, 'demand': 0}, {'id': 14, 'x': 36, 'y': 16, 'demand': 0}, {'id': 15, 'x': 52, 'y': 41, 'demand': 0}, {'id': 16, 'x': 27, 'y': 23, 'demand': 0}, {'id': 17, 'x': 17, 'y': 33, 'demand': 0}, {'id': 18, 'x': 13, 'y': 13, 'demand': 0}, {'id': 19, 'x': 57, 'y': 58, 'demand': 0}, {'id': 20, '

In [13]:
vehicle_capacity=40

In [3]:
def distance(a, b):
    return sqrt((a["x"] - b["x"])**2 + (a["y"] - b["y"])**2)


In [4]:
def create_individual():
    # توزيع العملاء عشوائياً على المخازن
    assignment = [random.choice([0, 1]) for _ in customers]
    
    # ترتيب عشوائي للعملاء
    routes = list(range(len(customers)))
    random.shuffle(routes)
    
    return {"assignment": assignment, "routes": routes}


In [5]:
def fitness(individual):
    total_distance = 0
    total_penalty = 0

    # فصل العملاء حسب كل depot
    for depot_id in range(len(depots)):
        assigned_customers = [
            idx for idx, a in enumerate(individual["assignment"]) if a == depot_id
        ]
        route = [i for i in individual["routes"] if i in assigned_customers]

        # حساب الطلب الكلي
        total_demand = sum(customers[i]["demand"] for i in route)
        if total_demand > vehicle_capacity:
            total_penalty += 1000  # عقوبة إذا تعدينا السعة

        # حساب المسافة المقطوعة ذهاباً وإياباً
        depot = depots[depot_id]
        prev = depot
        for i in route:
            total_distance += distance(prev, customers[i])
            prev = customers[i]
        total_distance += distance(prev, depot)  # العودة للمخزن

    return -(total_distance + total_penalty)  # عكس لأنه minimization


In [6]:
def selection(population):
    return max(random.sample(population, 3), key=fitness)


In [7]:
def crossover(p1, p2):
    cut = random.randint(1, len(p1["routes"]) - 1)
    child_routes = p1["routes"][:cut] + [r for r in p2["routes"] if r not in p1["routes"][:cut]]
    child_assignment = p1["assignment"][:]
    return {"assignment": child_assignment, "routes": child_routes}


In [8]:
def mutate(individual, rate=0.1):
    for i in range(len(individual["routes"])):
        if random.random() < rate:
            j = random.randint(0, len(individual["routes"]) - 1)
            individual["routes"][i], individual["routes"][j] = individual["routes"][j], individual["routes"][i]


In [16]:
def genetic_algorithm(generations=30, population_size=20):
    population = [create_individual() for _ in range(population_size)]

    for gen in range(generations):
        new_pop = []
        for _ in range(population_size):
            parent1 = selection(population)
            parent2 = selection(population)
            child = crossover(parent1, parent2)
            mutate(child)
            new_pop.append(child)

        population = new_pop
        best = max(population, key=fitness)
        print(f"Generation {gen+1}: Best Fitness = {-fitness(best):.2f}")

    best = max(population, key=fitness)
    print("Final Best Solution:")
    print("Assignments:", best["assignment"])
    print("Route:", best["routes"])


In [17]:
genetic_algorithm(generations=100,population_size=50)

Generation 1: Best Fitness = 1483.08
Generation 2: Best Fitness = 1474.60
Generation 3: Best Fitness = 1443.12
Generation 4: Best Fitness = 1412.09
Generation 5: Best Fitness = 1374.00
Generation 6: Best Fitness = 1407.03
Generation 7: Best Fitness = 1381.50
Generation 8: Best Fitness = 1393.45
Generation 9: Best Fitness = 1361.24
Generation 10: Best Fitness = 1389.96
Generation 11: Best Fitness = 1391.53
Generation 12: Best Fitness = 1415.76
Generation 13: Best Fitness = 1410.88
Generation 14: Best Fitness = 1445.50
Generation 15: Best Fitness = 1459.29
Generation 16: Best Fitness = 1450.27
Generation 17: Best Fitness = 1396.03
Generation 18: Best Fitness = 1402.95
Generation 19: Best Fitness = 1340.05
Generation 20: Best Fitness = 1350.02
Generation 21: Best Fitness = 1273.88
Generation 22: Best Fitness = 1344.64
Generation 23: Best Fitness = 1376.61
Generation 24: Best Fitness = 1385.13
Generation 25: Best Fitness = 1383.30
Generation 26: Best Fitness = 1396.86
Generation 27: Best F